In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
data = pd.read_csv("7Train1.csv", header=None)
bandwidth = data.iloc[:, 0].values  # Convert to NumPy array
length = len(bandwidth)

# Convert to PyTorch tensor
data_tensor = torch.FloatTensor(bandwidth).view(-1, 1)

# Function to create in-out sequences
def create_inout_sequences(input_data, window_size):
    inout_seq = []
    L = len(input_data)
    for i in range(L - window_size):
        train_seq = input_data[i:i + window_size]
        train_label = input_data[i + window_size:i + window_size + 1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Parameters
window_size = 5

# Create sequences for training from the first 3000 data points and last part after 3992
train_inout_seq = create_inout_sequences(data_tensor[982:], window_size) 

# Create sequences for testing from 3000 to 3992
test_inout_seq = create_inout_sequences(data_tensor[0:982], window_size)

# Define LSTM model with adjustable number of layers and dropout
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=128, output_size=1, num_layers=2, dropout=0.5):
        super(LSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, dropout=dropout)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq, hidden_state):
        lstm_out, hidden_state = self.lstm(input_seq, hidden_state)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1], hidden_state

    def init_hidden(self):
        return (torch.zeros(self.num_layers, 1, self.hidden_layer_size),
                torch.zeros(self.num_layers, 1, self.hidden_layer_size))

# Initialize the model, loss function, and optimizer
model = LSTM(dropout=0.6, num_layers=2)  # Adjust the dropout rate and number of layers as needed
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
epochs = 15
train_losses = []

for epoch in range(epochs):
    hidden_state = model.init_hidden()  # Initialize hidden state for each epoch

    epoch_train_loss = 0
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()

        seq = seq.view(-1, 1, 1)  # Reshape for LSTM input
        labels = labels.view(-1, 1)

        y_pred, hidden_state = model(seq, hidden_state)

        # Detach hidden state to prevent backpropagating through the entire history
        hidden_state = (hidden_state[0].detach(), hidden_state[1].detach())

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

        epoch_train_loss += single_loss.item()

    train_losses.append(epoch_train_loss / len(train_inout_seq))

    if epoch % 5 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {single_loss.item():.8f}')

# Making predictions
model.eval()
hidden_state = model.init_hidden()

predictions = []
for seq, _ in test_inout_seq:
    seq = seq.view(-1, 1, 1)  # Reshape for LSTM input
    with torch.no_grad():
        y_pred, hidden_state = model(seq, hidden_state)
        hidden_state = (hidden_state[0].detach(), hidden_state[1].detach())
        predictions.append(y_pred.item())

# Convert predictions to numpy array
predictions = np.array(predictions)

# Calculate actual values
actual_values = bandwidth[window_size:982]

# Calculate MAE and RMSE for all test data
mae = mean_absolute_error(actual_values, predictions)
rmse = np.sqrt(mean_squared_error(actual_values, predictions))

# Calculate error ratio as defined
mean_actual = np.mean(actual_values)
error_ratio_rmse = (rmse / mean_actual) * 100
error_ratio_mae = (mae / mean_actual) * 100

print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Error Ratio RMSE: {error_ratio_rmse:.4f}%')
print(f'Error Ratio MAE: {error_ratio_mae:.4f}%')

# Slice to get some 195 values for plotting
predictions_195 = predictions[:195]
actual_values_195 = actual_values[:195]

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(range(3000 + window_size, 3000 + window_size + 195), actual_values_195, label='Actual Data')
plt.plot(range(3000 + window_size, 3000 + window_size + 195), predictions_195, label='Predicted Data')
plt.legend()
plt.xlabel("Index")
plt.ylabel("Bandwidth")
plt.title("LSTM Predictions vs Actual Data (First 195 values from 3000 onwards)")
plt.show()
